### Import Required Libraries and Set Up Environment Variables

In [470]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [471]:
# Set the base URL to NASA's DONKI API:
BASE_URL = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{BASE_URL}CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [472]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)

# Check if the request was successful
if cme_response.status_code == 200:
    print("CME request successful!")
else:
    print(f"Request failed with status code {cme_response.status_code}")
    print("Error message:", cme_response.text)


CME request successful!


In [473]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [474]:
# Preview ONLY the first element from the cme_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))


{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [475]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.json_normalize(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
print(cme_df.head())

                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z   
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z   
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z   
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   

                                      linkedEvents  
0  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
1                                             None  
2                                             None  
3                                             None  
4  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  


In [476]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset=['linkedEvents'])

In [477]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    # Iterate over each dictionary in the list
     activity_id = cme_df.loc[i, 'activityID']
     start_time = cme_df.loc[i, 'startTime']
     linked_events = cme_df.loc[i, 'linkedEvents']
     # Iterate over each dictionary in the linkedEvents list
     for event in linked_events:
          
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
            'activityID': activity_id,
            'startTime': start_time,
            'linkedEvents': event
        })
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
print(expanded_df.head())

                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                    linkedEvents  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  


In [478]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict.get('activityID', None)  # Extract the activityID safely
    except (AttributeError, TypeError) as e:
        print(f"Error extracting activityID: {e}")  # Log the error for debugging
        return None  # Return None if an error occurs
        # Log the error or print it for debugging

# Test the function using one row from linkedEvents
test_activity_id = extract_activityID_from_dict(expanded_df.loc[0, 'linkedEvents'])
print("Extracted Activity ID:", test_activity_id)


Extracted Activity ID: 2013-05-04T04:52:00-IPS-001


In [479]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df.loc[:, 'GST_ActivityID'] = expanded_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

In [480]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df = expanded_df.dropna(subset=['GST_ActivityID'])

In [481]:
# print out the datatype of each column in this DataFrame:
expanded_df.dtypes

activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object

In [482]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df.loc[:, 'GST_ActivityID'] = expanded_df['GST_ActivityID'].astype(str)
# Convert startTime to datetime format  
expanded_df['startTime'] = pd.to_datetime(expanded_df['startTime'])
# Rename startTime to startTime_CME and activityID to cmeID
expanded_df = expanded_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'})
# Drop linkedEvents
expanded_df = expanded_df.drop(columns=['linkedEvents'])
# Verify that all steps were executed correctly
print(expanded_df.dtypes)  # Check data types
print(expanded_df.head())  # Check first few rows

cmeID                          object
startTime_CME     datetime64[ns, UTC]
GST_ActivityID                 object
dtype: object
                         cmeID             startTime_CME  \
0  2013-05-01T03:12:00-CME-001 2013-05-01 03:12:00+00:00   
1  2013-05-03T22:36:00-CME-001 2013-05-03 22:36:00+00:00   
2  2013-05-09T19:29:00-CME-001 2013-05-09 19:29:00+00:00   
3  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   
4  2013-05-13T02:54:00-CME-001 2013-05-13 02:54:00+00:00   

                GST_ActivityID  
0  2013-05-04T04:52:00-IPS-001  
1  2013-05-07T04:37:00-IPS-001  
2  2013-05-12T23:30:00-IPS-001  
3  2013-05-13T01:53:00-FLR-001  
4  2013-05-13T04:12:00-SEP-001  


In [483]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_df = expanded_df[expanded_df['GST_ActivityID'].str.contains('GST', na=False)]


### GST Data

In [484]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [485]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [486]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

In [487]:
# Preview ONLY the first element from the gst_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [488]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.json_normalize(gst_json)
# Keep only the columns: gstID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
# print(gst_df.head())

In [489]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=['linkedEvents'])
# print(gst_df.head())

In [490]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)
gst_df = gst_df.dropna(subset=['linkedEvents'])
print(gst_df.head())

                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  


In [491]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df.loc[:, 'CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df.dropna(subset=['CME_ActivityID'])
print(gst_df.head())

                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001  


In [492]:
print(gst_df.head())
# Convert the 'CME_ActivityID' column to string format 
gst_df.loc[:, 'CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_df.loc[:, 'gstID'] = gst_df['gstID'].astype(str)
# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])
# Rename startTime to startTime_GST 
gst_df = gst_df.rename(columns={'startTime': 'startTime_GST'})
# Drop linkedEvents
gst_df = gst_df.drop(columns=['linkedEvents'])
# Verify that all steps were executed correctly
print(gst_df.dtypes)
print(gst_df.head())

                         gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001  
gstID                          object
startTime_GST     datetime64[ns, UTC]
CME_ActivityID                 object
dtype: object
                         gstID             startTime_GST  \
0  2013-06-01

In [493]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME', na=False)]

### Merge both datatsets

In [494]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = gst_df.merge(
    expanded_df, 
    left_on=['gstID', 'CME_ActivityID'], 
    right_on=['GST_ActivityID', 'cmeID'], 
    how='inner'  # Ensures we only keep matching records
)
print(merged_df.head())

                         gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001  
2 2013-12-04 23:12:00+00:00  2013-12-08T00:00:00-GST-001  
3 2014-02-16 14:15:00+00

In [495]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"Rows in CME DataFrame: {expanded_df.shape[0]}")
print(f"Rows in GST DataFrame: {gst_df.shape[0]}")
print(f"Rows in Merged DataFrame: {merged_df.shape[0]}")


Rows in CME DataFrame: 61
Rows in GST DataFrame: 61
Rows in Merged DataFrame: 61


### Computing the time it takes for a CME to cause a GST

In [496]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df.loc[:, 'timeDiff'] = (merged_df['startTime_GST'] - merged_df['startTime_CME']).dt.total_seconds()

In [497]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
print(merged_df['timeDiff'].describe())

count        61.000000
mean     257366.557377
std       85989.336914
min      117360.000000
25%      187200.000000
50%      244260.000000
75%      308100.000000
max      529200.000000
Name: timeDiff, dtype: float64


### Exporting data in csv format

In [498]:
# Export data to CSV without the index
merged_df.to_csv('final_data.csv', index=False)

print("Final data exported to final_data.csv")

Final data exported to final_data.csv
